# 🌾 AgriTech - Crop Recommendation Demo

This notebook demonstrates how to use the **Crop Recommendation Model** from the AgriTech project.

It predicts the most suitable crop to grow based on:
- Soil nutrients (N, P, K)
- Environmental conditions (temperature, humidity, rainfall)
- Soil pH

---
**GitHub:** https://github.com/omroy07/AgriTech

## 📦 Step 1: Install Dependencies

In [ ]:
# Install required libraries
!pip install numpy pandas scikit-learn matplotlib seaborn requests joblib

## 📚 Step 2: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import warnings
warnings.filterwarnings('ignore')

print('✅ All libraries imported successfully!')

## 🔌 Step 3: Option A - Call the AgriTech API (Local Server)

In [ ]:
# -------------------------------------------------
# Make sure the AgriTech Flask server is running:
# cd AgriTech && python app.py
# -------------------------------------------------

BASE_URL = 'http://localhost:5000'

def predict_crop_api(N, P, K, temperature, humidity, ph, rainfall):
    """
    Call the AgriTech Crop Recommendation API.
    Returns the recommended crop as a string.
    """
    payload = {
        'N': N,
        'P': P,
        'K': K,
        'temperature': temperature,
        'humidity': humidity,
        'ph': ph,
        'rainfall': rainfall
    }
    try:
        response = requests.post(f'{BASE_URL}/predict_crop', json=payload, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.ConnectionError:
        return {'error': 'Server not running. Use Option B below to load the model directly.'}
    except Exception as e:
        return {'error': str(e)}

# Test the API
result = predict_crop_api(
    N=90, P=42, K=43,
    temperature=20.87, humidity=82.00,
    ph=6.50, rainfall=202.93
)
print('API Response:', result)

## 🤖 Step 4: Option B - Load and Use the Model Directly

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import os

# Try to load the pre-trained model from the repo
# Adjust the path if your model file is elsewhere
MODEL_PATHS = [
    '../Crop Recommendation/RandomForest.pkl',
    '../Crop Recommendation/model.pkl',
    'RandomForest.pkl',
    'model.pkl'
]

model = None
for path in MODEL_PATHS:
    if os.path.exists(path):
        model = joblib.load(path)
        print(f'✅ Model loaded from: {path}')
        break

if model is None:
    print('⚠️ Pre-trained model not found. Training a demo model using sample data...')
    print('   (For full accuracy, use the model from the Crop Recommendation folder)')

## 📊 Step 5: Load Sample Dataset and Train Demo Model

In [ ]:
# Sample crop recommendation dataset (subset)
# Full dataset: https://www.kaggle.com/datasets/atharvaingle/crop-recommendation-dataset

data = {
    'N':           [90, 85, 60, 74, 78, 20, 38, 100, 65, 45, 30, 25, 50, 70, 80],
    'P':           [42, 58, 55, 35, 42, 75, 67,  50, 60, 45, 60, 48, 40, 30, 55],
    'K':           [43, 41, 44, 40, 42, 25, 35,  50, 48, 38, 45, 55, 42, 28, 60],
    'temperature': [20.87, 21.77, 23.00, 26.49, 20.13, 37.50, 27.31, 22.50, 35.20, 25.50, 29.10, 28.60, 22.00, 26.80, 24.30],
    'humidity':    [82.00, 80.31, 82.32, 80.15, 81.60, 94.50, 68.32, 85.40, 92.00, 78.20, 88.50, 65.40, 80.20, 72.30, 76.50],
    'ph':          [6.50,  7.03,  7.84,  6.98,  7.63,  5.80,  6.90,  7.20,  5.50,  6.70,  7.10,  6.30,  7.50,  6.80,  6.10],
    'rainfall':    [202.93, 226.65, 263.96, 242.86, 245.73, 92.00, 150.50, 289.10, 120.00, 175.40, 220.00, 135.50, 195.00, 110.00, 260.00],
    'label':       ['rice','rice','jute','maize','chickpea','cotton','kidneybeans','rice','pigeonpeas','maize','blackgram','lentil','chickpea','mungbean','rice']
}

df = pd.DataFrame(data)
print('📊 Dataset shape:', df.shape)
print('\n🌿 Crops in dataset:', df['label'].unique())
df.head(10)

In [ ]:
# Train a simple demo model if no pre-trained model was found
if model is None:
    X = df.drop('label', axis=1)
    y = df['label']

    # For demo: use all data (normally split train/test)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_scaled, y)
    print('✅ Demo model trained successfully!')
    print('   Note: This is a demo model with limited data.')
    print('   For production use, refer to the Crop Recommendation folder.')

## 🌿 Step 6: Make a Single Prediction

In [ ]:
def predict_crop(N, P, K, temperature, humidity, ph, rainfall, model, scaler=None):
    """
    Predict the best crop for given soil and climate conditions.
    
    Parameters:
        N            : Nitrogen content (kg/ha)
        P            : Phosphorous content (kg/ha)
        K            : Potassium content (kg/ha)
        temperature  : Temperature in Celsius
        humidity     : Relative humidity (%)
        ph           : Soil pH (0-14)
        rainfall     : Rainfall in mm
    
    Returns:
        str: Recommended crop name
    """
    features = np.array([[N, P, K, temperature, humidity, ph, rainfall]])
    
    if scaler:
        features = scaler.transform(features)
    
    prediction = model.predict(features)
    probability = model.predict_proba(features).max() * 100
    return prediction[0], round(probability, 2)

# --- Example 1: Typical paddy field conditions ---
crop, confidence = predict_crop(
    N=90, P=42, K=43,
    temperature=20.87,
    humidity=82.00,
    ph=6.50,
    rainfall=202.93,
    model=model
)
print(f'\n🌾 Example 1 - Paddy Field Conditions')
print(f'   Recommended Crop : {crop.upper()}')
print(f'   Confidence       : {confidence}%')

# --- Example 2: Dry weather, high temperature ---
crop2, confidence2 = predict_crop(
    N=20, P=75, K=25,
    temperature=37.50,
    humidity=94.50,
    ph=5.80,
    rainfall=92.00,
    model=model
)
print(f'\n🌵 Example 2 - Dry/Hot Conditions')
print(f'   Recommended Crop : {crop2.upper()}')
print(f'   Confidence       : {confidence2}%')

## 📦 Step 7: Batch Predictions from JSON File

In [ ]:
# Load the sample JSON file from the examples folder
try:
    with open('sample_crop_input.json', 'r') as f:
        sample_data = json.load(f)

    print(f'✅ Loaded {len(sample_data["batch_predictions"])} samples from sample_crop_input.json')
    print('\n📋 Running Batch Predictions:')
    print('-' * 70)
    print(f'{"ID":<5} {"Expected":<15} {"Predicted":<15} {"Confidence":<12} {"Match"}')
    print('-' * 70)

    correct = 0
    for sample in sample_data['batch_predictions']:
        predicted, conf = predict_crop(
            sample['N'], sample['P'], sample['K'],
            sample['temperature'], sample['humidity'],
            sample['ph'], sample['rainfall'],
            model=model
        )
        match = '✅' if predicted == sample['expected_crop'] else '❌'
        if predicted == sample['expected_crop']:
            correct += 1
        print(f"{sample['id']:<5} {sample['expected_crop']:<15} {predicted:<15} {conf:<12} {match}")

    print('-' * 70)
    print(f'\nAccuracy on sample batch: {correct}/{len(sample_data["batch_predictions"])} = {correct/len(sample_data["batch_predictions"])*100:.1f}%')

except FileNotFoundError:
    print('⚠️ sample_crop_input.json not found. Make sure it is in the same folder as this notebook.')

## 📈 Step 8: Visualize Feature Importance

In [ ]:
feature_names = ['N (Nitrogen)', 'P (Phosphorous)', 'K (Potassium)',
                 'Temperature', 'Humidity', 'pH', 'Rainfall']

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10, 5))
colors = ['#2E8B57', '#3CB371', '#66CDAA', '#90EE90', '#228B22', '#006400', '#32CD32']
bars = plt.bar(range(len(feature_names)), importances[indices], color=colors)
plt.xticks(range(len(feature_names)), [feature_names[i] for i in indices], rotation=15, ha='right')
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.title('🌾 AgriTech - Feature Importance for Crop Recommendation')
plt.tight_layout()
plt.savefig('crop_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()
print('📊 Chart saved as crop_feature_importance.png')

## 🧪 Step 9: Interactive Prediction (Try Your Own Values!)

In [ ]:
# ✏️ EDIT THESE VALUES to test your own soil/climate conditions

my_N           = 78       # Nitrogen (kg/ha) - typical range: 0-140
my_P           = 42       # Phosphorous (kg/ha) - typical range: 5-145
my_K           = 42       # Potassium (kg/ha) - typical range: 5-205
my_temperature = 25.5     # Temperature (°C) - typical range: 8-44
my_humidity    = 71.0     # Humidity (%) - typical range: 14-100
my_ph          = 6.8      # Soil pH - typical range: 3.5-10
my_rainfall    = 150.0    # Rainfall (mm) - typical range: 20-300

# ------- Run the prediction -------
predicted_crop, confidence = predict_crop(
    my_N, my_P, my_K,
    my_temperature, my_humidity,
    my_ph, my_rainfall,
    model=model
)

print('=' * 45)
print('       🌱 CROP RECOMMENDATION RESULT')
print('=' * 45)
print(f'  Nitrogen      : {my_N} kg/ha')
print(f'  Phosphorous   : {my_P} kg/ha')
print(f'  Potassium     : {my_K} kg/ha')
print(f'  Temperature   : {my_temperature}°C')
print(f'  Humidity      : {my_humidity}%')
print(f'  Soil pH       : {my_ph}')
print(f'  Rainfall      : {my_rainfall} mm')
print('-' * 45)
print(f'  ✅ RECOMMENDED  : {predicted_crop.upper()}')
print(f'  📊 Confidence   : {confidence}%')
print('=' * 45)